Le but de ce fichier est de créer l'environnement fonctionnel le plus simple possible.
https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/


In [1]:
import gym
from gym.spaces import Discrete, Box
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers
import keras
from stable_baselines3 import DQN
import matplotlib.pyplot as plt
from stable_baselines3.common.torch_layers import BaseFeaturesExtractor

Solution pour le passage en live : mettre en argument de la classe des fonctions d'achats correspondant à celles définies dans bot_binance.

In [ ]:
def reward():
    #todo benji

In [2]:
class TradingEnv(gym.Env):
    def __init__(self, df, window_size=5, tc=20) :
        self.index=window_size
        self.window_size=window_size
        self.state=df[self.index-self.window_size:self.index]
        self.tc=tc
        #self.state=tf.Tensor(df[self.index-self.window_size:self.index])
        self.df=df
        self.holding=False
        self.action_space=Discrete(3)
        self.observation_space=Box(low=0, high=100000, shape=(5,))
        self.total_reward=[]


    def step(self, action):
        reward=0
        #action : provided by agent given the state
        price=self.df[self.index]
        prev_price=self.df[self.index-1]
        if action==0 : #buy
            #print('buy')
            reward=-price-self.tc
            self.holding=True
        elif action==1 and self.holding==True : #hold
            #print('hold')
            reward=price-prev_price
        elif action==2 and self.holding==True : #sell
            #print('sell')
            reward=price-self.tc
        
        self.index+=1
        self.state=self.df[self.index-self.window_size:self.index]
        
        done=False
        if self.index>400: #va définir la durée d'un épisode. A voir si il y a pas une mnière plus intelligente de faire
            done=True
        info={}

        self.total_reward.append((np.sum(self.total_reward)+reward))
        #print(len(self.total_reward))
        self.state=tf.reshape(self.state, shape=(5,))

        return self.state, reward, done, info

    def reset(self):
        self.index=self.window_size
        self.state=df[self.index-self.window_size:self.index]
        self.total_reward=[]
        return self.state

    def render(self):
        pass
        #todo

chargement des données d'entrainement. Obsolète avec le script d'extraction

In [3]:
prices=pd.read_csv("~/Developer/Trading bots/Trading bot crypto/Data/HitBTC_ETHUSD_1h.csv", sep=",")
prices = prices.rename(columns={'Close': 'value'})
df=prices['value']
df=pd.DataFrame({'value':df.values})
df=df['value']
env=TradingEnv(df=df)

In [4]:
agt=DQN('MlpPolicy', env, verbose=1)

Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [5]:
agt.learn(total_timesteps=25000)

-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 396       |
|    ep_rew_mean      | -1.42e+03 |
|    exploration_rate | 0.398     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 9069      |
|    time_elapsed     | 0         |
|    total_timesteps  | 1584      |
-----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 396      |
|    ep_rew_mean      | -2.6e+03 |
|    exploration_rate | 0.05     |
| time/               |          |
|    episodes         | 8        |
|    fps              | 9353     |
|    time_elapsed     | 0        |
|    total_timesteps  | 3168     |
----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 396       |
|    ep_rew_mean      | -1.06e+03 |
|    exploration_rate | 0.05      |
| time/               |           |
|  

doc stable baselines : https://stable-baselines.readthedocs.io/en/master/modules/dqn.html

In [6]:
agt.save('trading agent')
#agt=DQN.load('trading agent')

Cellule de test de l'environnement. Pourra éventuellement être utilisée pour le live trading également. Attention ça tourne à l'infini.

In [7]:
obs = env.reset()
while True:
    action, _states = agt.predict(obs, deterministic=True) #que fait predict? Comme le predict d'un NN? Aussi, quelle nuance entre obs et state?
    obs, reward, done, info = env.step(action)
    env.render()
    if done:
      obs = env.reset()

KeyboardInterrupt: 

pb on a que du hold ou sell ou buy ça alterne jamais lors d'un même episode.

# Custom policy

https://stable-baselines3.readthedocs.io/en/master/guide/custom_policy.html

In [ ]:
class CustomRNN(BaseFeaturesExtractor):
    #todo jules